Initialize notebook for Spark

In [1]:
import findspark
findspark.init()

Import required librries

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

Initialize SparkContext and SparkSession

In [3]:
sc=SparkContext.getOrCreate()
ss=SparkSession(sc)

Load Sample Data

In [ ]:
data=ss.read.csv("../data/titanic.csv", inferSchema=True, header=True)

Show Data Structure

In [ ]:
data.printSchema()

Show Data Types

In [ ]:
data.dtypes

Register a DataFrame as an SQL Temporary View

In [ ]:
data.createOrReplaceTempView('titanic')

Select 5 records

In [ ]:
sqldf=ss.sql("SELECT * FROM titanic limit 5").show()